In [1]:
import pandas as pd
import numpy as np
# from unipressed import IdMappingClient

In [3]:
#we ll parse the uniprotIDs from this, it is needed for the interaction file to build the db
prot = pd.read_csv ('protein_input.csv', index_col=False)

#list of intersaction pairs

combined = pd.read_csv ('cpdb_intersected(reformated).csv', index_col=False)

In [4]:
# ligand=list(combined.Ligand.values)
# receptor=list(combined.Receptor.values)

In [5]:
# request = IdMappingClient.submit(
#     source="GeneCards", dest="UniProtKB", ids=ligand
# )

In [6]:
# liglist=list(request.each_result())

In [7]:
# len(liglist)

In [8]:
combined["annotation_strategy"]="user_curated" #mandatory field
combined["source"]="user" #mandatory field
combined["partner_a"]=[prot.loc[prot['protein_name'] == x].uniprot.values[0] for x in combined["Ligand"]]#mandatory field
combined["partner_b"]=[prot.loc[prot['protein_name'] == x].uniprot.values[0] for x in combined["Receptor"]]#mandatory field

combined["protein_name_a"]=combined["Ligand"] #optional field
combined["protein_name_b"]=combined["Receptor"] #optional field

In [9]:
# PROC_PROCR  PROC_EPCR

In [10]:
prot.loc[prot['protein_name'] == "PROCR"]

,uniprot,protein_name,transmembrane,peripheral,secreted,secreted_desc,secreted_highlight,receptor,receptor_desc,integrin,...,pfam,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
1249,Q9UNN8,PROCR,True,False,False,NaN,False,True,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
combined.to_csv('combined_interactions.csv', index=False)

In [12]:
len(set(ligand+receptor))

NameError: name 'ligand' is not defined

In [ ]:
# below is the command used to create the db from terminal. 
# --user-interactions path to the interactions file
# --user-interactions-only, use only the submitted interactions, do not make corrections or add 
# these to the currentdb
# --result-path save the db to a specified path


# cellphonedb database generate --user-interactions combined.csv --user-interactions-only --result-path combined

In [ ]:
# Sanity check is done by investigating the SQLDB that is created by the cpdb. 

In [ ]:
# Sanity check

In [12]:
interaction = pd.read_csv ('./combined/sql_in_csv/interaction_table.csv', index_col=False)
interaction = interaction.iloc[:, [2,3]]

In [13]:
multi = pd.read_csv ('./combined/sql_in_csv/multidata_table.csv', index_col=None)
receptor = multi[multi['receptor'] == 1]
receptor = receptor.iloc[:, [0,1]]
ligand = multi[multi['receptor'] == 0]
ligand = ligand.iloc[:, [0,1]]

In [14]:
r=interaction[interaction['multidata_1_id'].isin(receptor['id_multidata'])].copy()
#fix column order for l-r
r[['multidata_1_id', 'multidata_2_id']] = r[['multidata_2_id', 'multidata_1_id']]
l=interaction[interaction['multidata_1_id'].isin(ligand['id_multidata'])].copy()

In [15]:
i=pd.concat([l, r], ignore_index=True).copy()

In [16]:
gene_table = pd.read_csv ('./combined/sql_in_csv/gene_table.csv', index_col=False)
gene_table = gene_table.iloc[:, [3,4]].copy()
protein_name = pd.read_csv ('./combined/sql_in_csv/protein_table.csv', index_col=False)
protein_name = protein_name.iloc[:, [1,5]].copy()
complex_comp = pd.read_csv ('./combined/sql_in_csv/complex_composition_table.csv', index_col=False)
complex_comp = complex_comp.iloc[:, [1,2]].copy()

In [17]:
i["ligand"] = ''
i["receptor"] = ''
i["l_adhesion_mol"] = ''
i["r_adhesion_mol"] = ''
i["l_complex"] = ''
i["r_complex"] = ''

In [18]:
df = pd.DataFrame(columns=['L', 'R'])
df["ligand"] = ''
df["receptor"] = ''
df["l_adhesion_mol"] = ''
df["r_adhesion_mol"] = ''

In [19]:
#tag adhesion molecules for rec
for x in range(len(i)):
    p_id = i.loc[x].multidata_1_id
    if p_id in receptor.id_multidata.values:
        i.at[x, 'r_adhesion_mol'] = 1
    else:
        i.at[x, 'r_adhesion_mol'] = 0
        
        
#tag adhesion molecules for lig
for x in range(len(i)):
    p_id = i.loc[x].multidata_2_id
    if p_id in ligand.id_multidata.values:
        i.at[x, 'l_adhesion_mol'] = 1
    else:
        i.at[x, 'l_adhesion_mol'] = 0
        

# assign gene symbols for ligand /// assign prot_names not gene symbols.
df = pd.DataFrame()
for x in range(len(i)):
    p_id = i.loc[x].multidata_1_id
    if p_id < 1282:
        symbol = gene_table.loc[gene_table['protein_id'] == p_id].hgnc_symbol.unique()
        p_name = protein_name.loc[protein_name['protein_multidata_id'] == p_id].protein_name
        p_name = p_name.str.replace('_HUMAN', '')
        i.at[x, 'ligand'] = list(p_name)[0]
        i.at[x, 'l_complex'] = 0
        #get those ligands that have more than one symbol in a seperate df
        #this part can be taken out as no need anymore?
        if len(symbol) > 1:
            entry=i.loc[x].to_frame().transpose()
            df = pd.concat([df,entry])
            df.at[x, 'ligand'] = symbol
    else:
        complex_pid = complex_comp.loc[complex_comp['complex_multidata_id'] == p_id].protein_multidata_id.values
        i.at[x, 'l_complex'] = 1
        #complex ids matching 1 or more than 1 need to be mapped
        if len(complex_pid) == 1:
            p_name = protein_name.loc[protein_name['protein_multidata_id'] == complex_pid[0]].protein_name
            p_name = p_name.str.replace('_HUMAN', '')
            i.at[x, 'ligand'] = list(p_name)[0]
            
        else:
            entry = list()
            for j in complex_pid:
                p_name = protein_name.loc[protein_name['protein_multidata_id'] == j].protein_name
                p_name = p_name.str.replace('_HUMAN', '')
                p_name = p_name.values[0]
                entry.append(p_name)
            i.at[x, 'ligand'] = '_'.join(entry)

In [20]:
for x in range(len(i)):
    p_id = i.loc[x].multidata_2_id
    if p_id < 1282:
        p_name = protein_name.loc[protein_name['protein_multidata_id'] == p_id].protein_name
        p_name = p_name.str.replace('_HUMAN', '')
        i.at[x, 'receptor'] = list(p_name)[0]
        i.at[x, 'r_complex'] = 0
    
    # complex mol
    else:
        complex_pid = complex_comp.loc[complex_comp['complex_multidata_id'] == p_id].protein_multidata_id.values
        i.at[x, 'r_complex'] = 1
        #complex ids matching 1 or more than 1 need to be mapped
        if len(complex_pid) == 1:
            p_name = protein_name.loc[protein_name['protein_multidata_id'] == complex_pid[0]].protein_name
            p_name = p_name.str.replace('_HUMAN', '')
            i.at[x, 'receptor'] = list(p_name)[0]
            
        else:
            entry = list()
            for j in complex_pid:
                p_name = protein_name.loc[protein_name['protein_multidata_id'] == j].protein_name
                p_name = p_name.str.replace('_HUMAN', '')
                p_name = p_name.values[0]
                entry.append(p_name)
            i.at[x, 'receptor'] = '_'.join(entry)

In [21]:
i["pairs"]=i[["ligand", "receptor"]].apply("_".join, axis=1)

In [22]:
i

,multidata_1_id,multidata_2_id,ligand,receptor,l_adhesion_mol,r_adhesion_mol,l_complex,r_complex,pairs
0,2,171,CCL24,CCR2,0,0,0,0,CCL24_CCR2
1,17,171,CCL16,CCR2,0,0,0,0,CCL16_CCR2
2,103,171,CCL2,CCR2,0,0,0,0,CCL2_CCR2
3,2,190,CCL24,CCR3,0,0,0,0,CCL24_CCR3
4,93,190,CCL3,CCR3,0,0,0,0,CCL3_CCR3
...,...,...,...,...,...,...,...,...,...
383,239,233,GAS6,MERTK,0,0,0,0,GAS6_MERTK
384,256,236,GDF6,BMPR2,0,0,0,0,GDF6_BMPR2
385,261,236,GDF7,BMPR2,0,0,0,0,GDF7_BMPR2
386,299,236,GDF2,BMPR2,0,0,0,0,GDF2_BMPR2


In [23]:
combined

,L_cpID,R_cpID,Ligand,Receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2,PairName,annotation_strategy,source,partner_a,partner_b,protein_name_a,protein_name_b
0,447,507,CXCL2,CXCR2,0,0,447,507,CXCL2_CXCR2,user_curated,user,P19875,P25025,CXCL2,CXCR2
1,448,507,CXCL3,CXCR2,0,0,448,507,CXCL3_CXCR2,user_curated,user,P19876,P25025,CXCL3,CXCR2
2,672,507,CXCL5,CXCR2,0,0,672,507,CXCL5_CXCR2,user_curated,user,P42830,P25025,CXCL5,CXCR2
3,799,507,CXCL6,CXCR2,0,0,799,507,CXCL6_CXCR2,user_curated,user,P80162,P25025,CXCL6,CXCR2
4,799,506,CXCL6,CXCR1,0,0,799,506,CXCL6_CXCR1,user_curated,user,P80162,P25024,CXCL6,CXCR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,308,1158,WNT2,FZD8,0,0,1373,308,WNT2_FZD8,user_curated,user,P09544,Q9H461,WNT2,FZD8
384,308,16,WNT2,FZD9,0,0,1375,308,WNT2_FZD9,user_curated,user,P09544,O00144,WNT2,FZD9
385,340,96,IAPP,RAMP3,0,0,340,1301,IAPP_RAMP3,user_curated,user,P10997,O60896,IAPP,RAMP3
386,308,98,WNT2,FZD7,0,0,1371,308,WNT2_FZD7,user_curated,user,P09544,O75084,WNT2,FZD7


In [24]:
[x for x in i.pairs.values if x not in combined.PairName.values]

['PLD2_ARF1',
 'ERBB4_NRG1',
 'EPHA4_EFNB2',
 'EPHB3_EFNB2',
 'EPHB4_EFNB2',
 'EPHB1_EFNB2']

In [25]:
combined.loc[combined['Ligand'] == "PLD2"]

,L_cpID,R_cpID,Ligand,Receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2,PairName,annotation_strategy,source,partner_a,partner_b,protein_name_a,protein_name_b


In [26]:
i.loc[i['pairs'] == "PLD2_ARF1"]

,multidata_1_id,multidata_2_id,ligand,receptor,l_adhesion_mol,r_adhesion_mol,l_complex,r_complex,pairs
67,13,219,PLD2,ARF1,1,0,0,0,PLD2_ARF1


In [27]:
combined.loc[combined['PairName'] == "PROC_EPCR"]

,L_cpID,R_cpID,Ligand,Receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2,PairName,annotation_strategy,source,partner_a,partner_b,protein_name_a,protein_name_b


In [28]:
combined.loc[combined['Ligand'] == "PROC"]

,L_cpID,R_cpID,Ligand,Receptor,l_adhesion_mol,r_adhesion_mol,pid1,pid2,PairName,annotation_strategy,source,partner_a,partner_b,protein_name_a,protein_name_b
291,223,1259,PROC,PROCR,0,0,223,1259,PROC_PROCR,user_curated,user,P04070,Q9UNN8,PROC,PROCR


In [29]:
# comparing it to raw to identify which edges are coming from complex mols. 

In [30]:
interaction_raw = pd.read_csv ('./cpdb_raw/interaction_table.csv', index_col=False)
interaction_raw = interaction_raw.iloc[:, [2,3]]
multi_raw = pd.read_csv ('./cpdb_raw/multidata_table.csv', index_col=None)
receptor_raw = multi_raw[multi_raw['receptor'] == 1]
receptor_raw = receptor_raw.iloc[:, [0,1]]
ligand_raw = multi_raw[multi_raw['receptor'] == 0]
ligand_raw = ligand_raw.iloc[:, [0,1]]
r_raw=interaction_raw[interaction_raw['multidata_1_id'].isin(receptor_raw['id_multidata'])].copy()
#fix column order for l-r
r_raw[['multidata_1_id', 'multidata_2_id']] = r_raw[['multidata_2_id', 'multidata_1_id']]
l_raw=interaction_raw[interaction_raw['multidata_1_id'].isin(ligand_raw['id_multidata'])].copy()
i_raw=pd.concat([l_raw, r_raw], ignore_index=True).copy()
gene_table_raw = pd.read_csv ('./cpdb_raw/gene_table.csv', index_col=False)
gene_table_raw = gene_table_raw.iloc[:, [3,4]].copy()
protein_name_raw = pd.read_csv ('./cpdb_raw/protein_table.csv', index_col=False)
protein_name_raw = protein_name_raw.iloc[:, [1,5]].copy()
complex_comp_raw = pd.read_csv ('./cpdb_raw/complex_composition_table.csv', index_col=False)
complex_comp_raw = complex_comp_raw.iloc[:, [1,2]].copy()
i_raw["ligand"] = ''
i_raw["receptor"] = ''
i_raw["l_adhesion_mol"] = ''
i_raw["r_adhesion_mol"] = ''
i_raw["l_complex"] = ''
i_raw["r_complex"] = ''
df = pd.DataFrame(columns=['L', 'R'])
df["ligand"] = ''
df["receptor"] = ''
df["l_adhesion_mol"] = ''
df["r_adhesion_mol"] = ''

#tag adhesion molecules for rec
for x in range(len(i_raw)):
    p_id = i_raw.loc[x].multidata_1_id
    if p_id in receptor_raw.id_multidata.values:
        i_raw.at[x, 'r_adhesion_mol'] = 1
    else:
        i_raw.at[x, 'r_adhesion_mol'] = 0
        
        
#tag adhesion molecules for lig
for x in range(len(i_raw)):
    p_id = i_raw.loc[x].multidata_2_id
    if p_id in ligand_raw.id_multidata.values:
        i_raw.at[x, 'l_adhesion_mol'] = 1
    else:
        i_raw.at[x, 'l_adhesion_mol'] = 0
        

# assign gene symbols for ligand /// assign prot_names not gene symbols.
df = pd.DataFrame()
for x in range(len(i_raw)):
    p_id = i_raw.loc[x].multidata_1_id
    if p_id < 1282:
        symbol = gene_table_raw.loc[gene_table_raw['protein_id'] == p_id].hgnc_symbol.unique()
        p_name = protein_name_raw.loc[protein_name_raw['protein_multidata_id'] == p_id].protein_name
        p_name = p_name.str.replace('_HUMAN', '')
        i_raw.at[x, 'ligand'] = list(p_name)[0]
        i_raw.at[x, 'l_complex'] = 0
        #get those ligands that have more than one symbol in a seperate df
        #this part can be taken out as no need anymore?
        if len(symbol) > 1:
            entry=i_raw.loc[x].to_frame().transpose()
            df = pd.concat([df,entry])
            df.at[x, 'ligand'] = symbol
    else:
        complex_pid = complex_comp_raw.loc[complex_comp_raw['complex_multidata_id'] == p_id].protein_multidata_id.values
        i_raw.at[x, 'l_complex'] = 1
        #complex ids matching 1 or more than 1 need to be mapped
        if len(complex_pid) == 1:
            p_name = protein_name_raw.loc[protein_name_raw['protein_multidata_id'] == complex_pid[0]].protein_name
            p_name = p_name.str.replace('_HUMAN', '')
            i_raw.at[x, 'ligand'] = list(p_name)[0]
            
        else:
            entry = list()
            for j in complex_pid:
                p_name = protein_name_raw.loc[protein_name_raw['protein_multidata_id'] == j].protein_name
                p_name = p_name.str.replace('_HUMAN', '')
                p_name = p_name.values[0]
                entry.append(p_name)
            i_raw.at[x, 'ligand'] = '_'.join(entry)
            
for x in range(len(i_raw)):
    p_id = i_raw.loc[x].multidata_2_id
    if p_id < 1282:
        p_name = protein_name_raw.loc[protein_name_raw['protein_multidata_id'] == p_id].protein_name
        p_name = p_name.str.replace('_HUMAN', '')
        i_raw.at[x, 'receptor'] = list(p_name)[0]
        i_raw.at[x, 'r_complex'] = 0
    
    # complex mol
    else:
        complex_pid = complex_comp_raw.loc[complex_comp_raw['complex_multidata_id'] == p_id].protein_multidata_id.values
        i_raw.at[x, 'r_complex'] = 1
        #complex ids matching 1 or more than 1 need to be mapped
        if len(complex_pid) == 1:
            p_name = protein_name_raw.loc[protein_name_raw['protein_multidata_id'] == complex_pid[0]].protein_name
            p_name = p_name.str.replace('_HUMAN', '')
            i_raw.at[x, 'receptor'] = list(p_name)[0]
            
        else:
            entry = list()
            for j in complex_pid:
                p_name = protein_name_raw.loc[protein_name_raw['protein_multidata_id'] == j].protein_name
                p_name = p_name.str.replace('_HUMAN', '')
                p_name = p_name.values[0]
                entry.append(p_name)
            i_raw.at[x, 'receptor'] = '_'.join(entry)
            
i_raw["pairs"]=i_raw[["ligand", "receptor"]].apply("_".join, axis=1)

In [31]:
len([x for x in i.pairs.values if x not in i_raw.pairs.values])

113

In [32]:
[x for x in i.pairs.values if x not in i_raw.pairs.values]

['WNT7A_LRP6',
 'WNT7A_FZD5',
 'WNT1_FZD5',
 'WNT2_FZD5',
 'WNT5A_FZD5',
 'WNT3_FZD5',
 'WNT3A_FZD5',
 'WNT7A_FZD10',
 'WNT7B_FZD10',
 'WNT1_RYK',
 'WNT1_FZD8',
 'WNT2_FZD8',
 'WNT5A_FZD8',
 'WNT3_FZD8',
 'WNT3A_FZD8',
 'WNT1_FZD3',
 'WNT1_FZD1',
 'WNT2_FZD1',
 'WNT3_FZD1',
 'WNT3A_FZD1',
 'WNT7B_FZD1',
 'WNT5A_FZD2',
 'WNT3A_FZD2',
 'WNT4_FZD2',
 'WNT5A_FZD4',
 'NDP_FZD4',
 'FGF19_FGFR4',
 'IHH_CDON',
 'SHH_CDON',
 'SHH_BOC',
 'GDF9_BMPR2',
 'BMP10_BMPR2',
 'BMP15_BMPR2',
 'BMP2_BMPR2',
 'BMP4_BMPR2',
 'BMP7_BMPR2',
 'BMP5_BMPR2',
 'BMP6_BMPR2',
 'GDF5_BMPR2',
 'BMP7_ACVR1',
 'BMP5_ACVR1',
 'BMP6_ACVR1',
 'INHA_ACVR1',
 'INHBA_ACVR1',
 'INHBB_ACVR1',
 'PSPN_GFRA4',
 'TSHB_TSHR',
 'FSHB_FSHR',
 'IL1A_IL1R1',
 'IL1B_IL1R1',
 'IL1A_IL1R2',
 'IL1B_IL1R2',
 'VWF_GP1BA',
 'IL4_IL2RG',
 'IL7_IL2RG',
 'IL9_IL2RG',
 'IL9_IL9R',
 'IL5_IL5RA',
 'IL3_IL3RA',
 'IAPP_CALCR',
 'LIF_LIFR',
 'CNTF_LIFR',
 'CNTF_CNTFR',
 'IL12B_IL23R',
 'GDNF_GFRA1',
 'IL21_IL21R',
 'WNT7A_FZD9',
 'WNT1_FZD9',
 'WNT1_F

In [34]:
i_raw.loc[i_raw['receptor'] == "WNT7A"]

,multidata_1_id,multidata_2_id,ligand,receptor,l_adhesion_mol,r_adhesion_mol,l_complex,r_complex,pairs
167,1357,38,LRP5_FZD10,WNT7A,1,0,1,0,LRP5_FZD10_WNT7A
168,1358,38,LRP6_FZD10,WNT7A,1,0,1,0,LRP6_FZD10_WNT7A
169,1359,38,LRP5_FZD1,WNT7A,1,0,1,0,LRP5_FZD1_WNT7A
170,1360,38,LRP6_FZD1,WNT7A,1,0,1,0,LRP6_FZD1_WNT7A
171,1361,38,LRP5_FZD2,WNT7A,1,0,1,0,LRP5_FZD2_WNT7A
172,1362,38,LRP6_FZD2,WNT7A,1,0,1,0,LRP6_FZD2_WNT7A
173,1363,38,LRP5_FZD3,WNT7A,1,0,1,0,LRP5_FZD3_WNT7A
174,1364,38,LRP6_FZD3,WNT7A,1,0,1,0,LRP6_FZD3_WNT7A
175,1365,38,LRP5_FZD4,WNT7A,1,0,1,0,LRP5_FZD4_WNT7A
176,1366,38,LRP6_FZD4,WNT7A,1,0,1,0,LRP6_FZD4_WNT7A


In [37]:
i.loc[i['pairs'] == "WNT7A_LRP6"]

,multidata_1_id,multidata_2_id,ligand,receptor,l_adhesion_mol,r_adhesion_mol,l_complex,r_complex,pairs
31,10,36,WNT7A,LRP6,0,0,0,0,WNT7A_LRP6


In [ ]:
pwd